In [1]:
#tu rzeczy do instalacji, zaczynac od !
#%pip install tensorflow
#%pip install matplotlib
#%pip install pandas
#tu sklearn, możliwe że trzeba zrestartowac krenel po updacie
#%pip install -U scikit-learn

In [2]:
#tu biblioteki
import numpy as np
import pandas as pd
import time
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras import regularizers
from matplotlib import pyplot as plt
from sklearn.metrics import roc_curve, auc, accuracy_score
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

print("Import done")

Num GPUs Available:  1
Import done


In [3]:
#GPUs - tu poki co nie dziala, trzeba postawic srodowisko wirtualne
gpus = tf.config.experimental.list_physical_devices('GPU')
print (len(gpus), " GPUs found!")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), " Physical GPUs, ", len(logical_gpus), " Logical GPUs")
    except RuntimeError as e:
        print(e)
print("GPUs done")

1  GPUs found!
1  Physical GPUs,  1  Logical GPUs
GPUs done


In [4]:
def directory_flow(train_directory, test_directory, val_directory, shape):
    #directory = nazwy folderow,
    #shape = rozmiar obrazka, u nas 256x256
    print ("generators start")
    image_generator=ImageDataGenerator()
    train_gen = image_generator.flow_from_directory(train_directory, target_size=shape, shuffle=True) #sciezka do train dir
    print ("train done")
    val_gen = image_generator.flow_from_directory(val_directory, target_size=shape, shuffle=True)
    print ("validate done")
    test_gen = image_generator.flow_from_directory(test_directory, target_size=shape, shuffle=True) #sciezka do test dir
    print ("generators done")
    #zastanowic sie nad zwracaniem validate gen
    return train_gen, test_gen, val_gen

In [5]:
#nie zadziala, bo jeszcze nie ma obrazkow
train_gen, test_gen, val_gen = directory_flow("./archive/TrainingData", "./archive/TestingData", "./archive/ValidationData", (224, 224))
print("Directory flow done!")

generators start
Found 30000 images belonging to 15 classes.
train done
Found 2000 images belonging to 15 classes.
validate done
Found 2407 images belonging to 15 classes.
generators done
Directory flow done!


In [6]:
#model
def make_model(shape):
    #model = Sequential()
    #model.add(Conv2D(kernel_size = (3,3), filters = 3, activation = "relu", input_shape=shape))
    #model.add(Conv2D(kernel_size = (3,3), filters = 3, activation = "relu"))
    #model.add(Flatten())
    #model.add(Dense(256, activation = "relu"))
    #model.add(Dense(15, activation = "softmax"))
    #model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
    #return model
    base_model = tf.keras.applications.xception.Xception(include_top=False, weights="imagenet",input_shape=shape, pooling='max')
    model = Sequential([
        base_model,
        BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
        Dense(256, kernel_regularizer= regularizers.l2(l= 0.016), activity_regularizer= regularizers.l1(0.006),
                    bias_regularizer= regularizers.l1(0.006), activation= 'relu'),
        Dropout(rate= 0.45, seed= 123),
        Dense(15, activation= 'softmax')
    ])

    model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])
    return model
print("Model done!")

Model done!


In [7]:
model=make_model((224, 224, 3))
print("Model generated")

Model generated


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 2048)              20861480  
_________________________________________________________________
batch_normalization_4 (Batch (None, 2048)              8192      
_________________________________________________________________
dense (Dense)                (None, 256)               524544    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 15)                3855      
Total params: 21,398,071
Trainable params: 21,339,447
Non-trainable params: 58,624
_________________________________________________________________


In [9]:
#zadziala jak wrzuce obrazki
model.fit(x=train_gen, epochs=40, validation_data=(val_gen),validation_steps= None, shuffle= False)

Epoch 1/40


ResourceExhaustedError:  OOM when allocating tensor with shape[32,128,109,109] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential/xception/block2_sepconv2_bn/FusedBatchNormV3 (defined at C:\Users\Zuza\AppData\Local\Temp\ipykernel_19164\3878720833.py:2) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_10851]

Function call stack:
train_function
